# TBRU Supervised Results

In [ ]:
import sys
sys.path.append("/data/srlab/lrumker/MCSC_Project/cna-display/")

In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import cna
import pp, pf
import pickle
plt.style.use('../pp.mplstyle')
np.random.seed(0) # for reproducibility

In [ ]:
from scipy import stats

# Export NAM PCs for Interpretation

### Harmonized 20 CCs

In [ ]:
np.random.seed(0) # for reproducibility

In [ ]:
print('reading')
d = cna.read(pf.tbru_h5ad + 'harmcca20.h5ad')
d.obs_to_sample(['batch', 'nUMI', 'percent_mito'])

In [ ]:
batches = d.samplem.batch.values

In [ ]:
# Save NAM PCs for Gene Set Analysis
res = cna.tl._association.association(d, y = d.samplem.TB_STATUS_CASE.values, 
                                      batches=batches, 
                                      covs=d.samplem[['nUMI', 'percent_mito']].values, 
                                      Nnull=10000,
                                     local_test = False)

In [ ]:
dummy_df = pd.DataFrame(d.uns['NAM_nbhdXpc'].iloc[:,0:20])
dummy_df.to_csv("/data/srlab/lrumker/MCSC_Project/TBRU_dset_extras/TBRU_cna_NAM_PCs.txt")

### Non-Harmonized mRNA

In [ ]:
np.random.seed(0) # for reproducibility

In [ ]:
print('reading')
d = cna.read(pf.tbru_h5ad + 'mrna.h5ad')
d.obs_to_sample(['batch', 'nUMI', 'percent_mito'])

In [ ]:
batches = d.samplem.batch.values
covs = d.samplem[['nUMI', 'percent_mito']].values

In [ ]:
# Save NAM PCs for Gene Set Analysis
res = cna.tl._association.association(d, y = d.samplem.TB_STATUS_CASE.values, 
                                      batches=batches, 
                                      covs=covs, 
                                      Nnull=10000,
                                     local_test = False)

In [ ]:
dummy_df = pd.DataFrame(d.uns['NAM_nbhdXpc'].iloc[:,0:20])
dummy_df.to_csv("/data/srlab/lrumker/MCSC_Project/TBRU_dset_extras/TBRU_cna_NAM_PCs_mrna.txt")

In [ ]:
# NAM PC 2 is positively correlated with male sex
np.corrcoef(d.samplem.Sex_M, d.uns['NAM_sampleXpc'].iloc[:,1])

# Plot Original Clusters

In [ ]:
palette = plt.get_cmap('tab20').colors

In [ ]:
clust_nums = [25,1,3,0,28,4,5,10,6,12,8,14,17,16,21,9,2,22,11,24,27,18,
              7,19,23,26,29,15,13,20,30,31]
clust_annotations = ['CD4+ CD38+ICOS+ central', 'CD4+ CD27+', 'CD4+ CCR4+ICOS+ central',
                       'CD4+ CD27+ Th17','CD4+ CTLA4+ Treg','CD4+ CCR4+ICOS+','CD4+ central',
                       'CD4+ lncRNA','CD4+ Treg','CD4+ CCR4+','CD4+ Th2','CD4+ Th17',
                       'CD4+ CXCR3+CCR6+ Th1/Th17','CD4+ CD161+ Th2','CD4+ activated',
                       'CD4+ CCR6+CXCR3+ Th1/Th17','CD4+ CD29+CXCR3+ Th1','CD4+ HLA-DR+',
                       'CD4+ CD161+ Th1','CD4+ CCR5+CCR6+ Th1/Th17','CD4+ CCR5+ cytotoxic',
                       'CD4+ CD161+ cytotoxic','CD4+ cytotoxic','CD4/8+ PD1+TIGIT+',
                       'CD8+ central','CD8+ CXCR3+','CD8+ activated','CD8+ GZMK','CD8+ GZMH+',
                       'DN/CD8+ gamma delta','DN SOX4+ gamma delta', 'CD4+ BARX1+']
cluster_annotations = {clust_nums[i]: clust_annotations[i] for i in range(len(clust_nums))} 

In [ ]:
text_xvals = []
text_yvals = []
text_labels = []
for i_cluster in d.obs['aparna2p0'].value_counts().index[0:32]:
    text_labels.append(cluster_annotations[i_cluster])
    loc_cluster = np.where(d.obs['aparna2p0']==i_cluster)[0]
    means = np.mean(d.obsm['X_umap'][loc_cluster,0:2], axis=0)
    text_xvals.append(means[0])
    text_yvals.append(means[1])

In [ ]:
#create a new figure
plt.figure(figsize=(5,5))

#loop through labels and plot each cluster
cc=0
for i_cluster in d.obs['aparna2p0'].value_counts().index[0:32]:
    loc_cluster = np.where(d.obs['aparna2p0']==i_cluster)[0]
    
    #add data points 
    plt.scatter(x=d.obsm['X_umap'][loc_cluster,0], 
                y=d.obsm['X_umap'][loc_cluster,1], 
                color="C"+str(cc), 
                alpha=1,
                s=0.2)
    
    #add label
    plt.annotate(i_cluster, 
                 np.mean(d.obsm['X_umap'][loc_cluster,0:2], axis=0),
                 horizontalalignment='center',
                 verticalalignment='center',
                 size=8, weight='bold',
                 color='white',
                 backgroundcolor="C"+str(cc))
    cc=cc+1

#texts = [plt.text(text_xvals[i], text_yvals[i], text_labels[i], 
#                      ha='center', va='center', fontsize = 5) for i in np.arange(len(text_labels))]
#adjust_text(texts)

# Survey for Global Associations

### Quality Control to Determine Phenotypes Tested

In [ ]:
# Extract all phenotypes
all_covars = d.samplem.columns
all_covars = np.array(all_covars)[np.array(['leiden' not in x for x in d.samplem.columns])]
all_covars = np.array(all_covars)[np.array(['aparna' not in x for x in all_covars])]
all_covars = np.array(all_covars)[np.array([x not in ['batch','C','donor','HH_id', 
                                                      'identifier', 'nUMI', 'percent_mito'] for x in all_covars])]
print(len(all_covars))
all_covars

In [ ]:
# Check for excessive missingness
nan_variables = np.array(all_covars)[np.array(['nan' in x for x in all_covars])]
np.sum(d.samplem[nan_variables], axis = 0)/d.samplem.shape[0]

In [ ]:
# Remove phenotypes with missingness > 10%
all_covars = np.array(all_covars)[np.array(['IPT' not in x for x in all_covars])]
all_covars

In [ ]:
# Identify variables containing nans
np.array(all_covars)[np.array(['nan' in x for x in all_covars])]

In [ ]:
# Add nan values to each phenotype
d.samplem['BCG_vaccine_1.0'].loc[(d.samplem['BCG_vaccine_nan']==1).values] = np.nan

d.samplem['sesgroup2_low'].loc[(d.samplem['sesgroup2_nan']==1).values] = np.nan
d.samplem['sesgroup2_medium'].loc[(d.samplem['sesgroup2_nan']==1).values] = np.nan
d.samplem['sesgroup2_high'].loc[(d.samplem['sesgroup2_nan']==1).values] = np.nan

d.samplem['smk_status_Heavy'].loc[(d.samplem['smk_status_nan']==1).values] = np.nan
d.samplem['smk_status_Light'].loc[(d.samplem['smk_status_nan']==1).values] = np.nan
d.samplem['smk_status_Non-Smoker'].loc[(d.samplem['smk_status_nan']==1).values] = np.nan

d.samplem['DRK_status_Heavy'].loc[(d.samplem['DRK_status_nan']==1).values] = np.nan
d.samplem['DRK_status_Light'].loc[(d.samplem['DRK_status_nan']==1).values] = np.nan
d.samplem['DRK_status_Non-Drinker'].loc[(d.samplem['DRK_status_nan']==1).values] = np.nan

d.samplem['malnutrition_overweight'].loc[(d.samplem['malnutrition_nan']==1).values] = np.nan
d.samplem['malnutrition_normal'].loc[(d.samplem['malnutrition_nan']==1).values] = np.nan
d.samplem['malnutrition_underweight'].loc[(d.samplem['malnutrition_nan']==1).values] = np.nan

all_covars = np.array(all_covars)[np.array(['nan' not in x for x in all_covars])]
all_covars

In [ ]:
# Remove one-hot categorical phenotypes with fewer than 20 cases
num_values = [len(np.unique(d.samplem[covar])[~np.isnan(np.unique(d.samplem[covar]))])\
              for covar in all_covars]
binary_vars = all_covars[np.array(num_values)==2]
counts = np.sum(d.samplem[binary_vars], axis = 0)
to_remove = counts.index[counts.values<20] # Also checked for variables with counts.values>(d.N-20); there are none
print(to_remove)
all_covars = np.array(all_covars)[np.array([x not in to_remove for x in all_covars])]
all_covars

In [ ]:
# Remove TB phenotype, since it is examined separately
to_remove = ['TB_STATUS_CASE']
all_covars = np.array(all_covars)[np.array([x not in to_remove for x in all_covars])]
all_covars

In [ ]:
# Heatmap of correlations among phenotypes
harvest = np.around(d.samplem[all_covars].corr()**2, 2)# R^2

fig, ax = plt.subplots(figsize=(10,7))
im = ax.imshow(harvest, cmap = "viridis")
ax.set_xticks(np.arange(len(all_covars)))
ax.set_yticks(np.arange(len(all_covars)))
ax.set_xticklabels(all_covars, fontsize = 7)
ax.set_yticklabels(all_covars, fontsize = 7)
plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

for i in range(len(all_covars)):
    for j in range(len(all_covars)):
        if(harvest.iloc[i, j] != 0.0):
            text = ax.text(j, i, harvest.iloc[i, j], ha="center", va="center", color="w", fontsize = 5)

ax.set_title("Correlations among TBRU phenotypes", fontsize=10)
fig.tight_layout()
plt.show()

In [ ]:
# Remove one phenotype from each pair correlated > 0.5  
to_remove = ['NATad4KR', 'tbru_age', 'BMI', 'sesgroup',
             'season_Summer',
             'DRK_status_Light', 'malnutrition_overweight']
all_covars = np.array(all_covars)[np.array([x not in to_remove for x in all_covars])]
all_covars

In [ ]:
# Heatmap of correlations among phenotypes
harvest = np.around(d.samplem[all_covars].corr()**2, 2)# R^2

fig, ax = plt.subplots(figsize=(10,7))
im = ax.imshow(harvest, cmap = "viridis")
ax.set_xticks(np.arange(len(all_covars)))
ax.set_yticks(np.arange(len(all_covars)))
ax.set_xticklabels(all_covars, fontsize = 7)
ax.set_yticklabels(all_covars, fontsize = 7)
plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

for i in range(len(all_covars)):
    for j in range(len(all_covars)):
        if(harvest.iloc[i, j] != 0.0):
            text = ax.text(j, i, harvest.iloc[i, j], ha="center", va="center", color="w", fontsize = 5)

ax.set_title("Correlations among TBRU phenotypes", fontsize=10)
fig.tight_layout()
plt.show()

In [ ]:
# Remove low-variance covariates with std < 0.1
covar_variance = np.std(d.samplem[all_covars],axis=0)
covar_variance = covar_variance[np.argsort(covar_variance)]
to_remove = covar_variance.index[covar_variance.values<0.1]
print(to_remove)
all_covars = np.array(all_covars)[np.array([x not in to_remove for x in all_covars])]
all_covars

In [ ]:
QCed_phenos = all_covars
QCed_phenos

## Covariate Selection

In [ ]:
selected_covars = pd.DataFrame(np.full((len(QCed_phenos), len(QCed_phenos)), False))
selected_covars.index = QCed_phenos
selected_covars.columns = QCed_phenos
selected_covars['TB_STATUS_CASE'] = np.repeat(False, selected_covars.shape[0])
for row_pheno in selected_covars.index:
    for col_pheno in selected_covars.columns:
        if row_pheno is not col_pheno:
            slope, intercept, r_value, p_value, std_err = stats.linregress(d.samplem[row_pheno].values,
                                                                       d.samplem[col_pheno].values)
            if p_value<0.05:
            #if p_value<(0.05/(len(QCed_phenos)+1)):
            #if p_value<(0.05/np.sqrt(len(QCed_phenos)+1)):
                i_row = np.where(selected_covars.index==row_pheno)[0][0]
                i_col = np.where(selected_covars.columns==col_pheno)[0][0]
                if i_row != i_col:
                    selected_covars.iloc[i_row,i_col]=True

In [ ]:
#num_nominal = np.sum(selected_covars, axis=1)

In [ ]:
#num_bonf = np.sum(selected_covars, axis=1)

In [ ]:
num_adj = np.sum(selected_covars, axis=1)

In [ ]:
p_nominal = global_ps

In [ ]:
p_bonf = global_ps

In [ ]:
p_adj = global_ps

In [ ]:
import statsmodels.stats.multitest as fdr

In [ ]:
fdr.fdrcorrection(p_nominal)[0]

In [ ]:
pd.DataFrame({ "Nom: #Covs": num_nominal, "Adj: #Covs": num_adj,"Bonf: #Covs": num_bonf, 
             "Nom: Sig": p_nominal<0.05/len(QCed_phenos),
              "Adj: Sig": p_adj<0.05/len(QCed_phenos),
             "Bonf: Sig": p_bonf<0.05/len(QCed_phenos),
             "Nom: FDR": fdr.fdrcorrection(p_nominal)[0],
             "Adj: FDR": fdr.fdrcorrection(p_adj)[0],
             "Bonf: FDR": fdr.fdrcorrection(p_bonf)[0] })

### Generate Cluster-Based Survey Results

In [ ]:
from methods import methods

In [ ]:
d.obs['aparna2p0_str'] = d.obs['aparna2p0'].astype(str)

In [ ]:
masc_globalp = []
poscells_masc = np.repeat(0, d.obs.shape[0])
negcells_masc = np.repeat(0, d.obs.shape[0])
batches = d.samplem.batch.values
for i_pheno in np.arange(len(QCed_phenos)):
    sel_covars = selected_covars.iloc[i_pheno,:]
    covs = d.samplem[sel_covars.index[sel_covars]].values
    res = methods._MASC(d, d.samplem[QCed_phenos[i_pheno]].values, batches, d.samplem.C.values, 
                        covs, # sample-level covariates                                                                                                                                                      
                        d.obs[['nUMI', 'percent_mito']].values, # Cell-level covariates
                        clustertype='aparna2p0_str')
    betas = res[3][0]
    clust_ps = res[3][1]
    n_clust = len(clust_ps)
    posclust = np.where(((clust_ps*n_clust)<0.05)*(betas>0))[0]
    negclust = np.where(((clust_ps*n_clust)<0.05)*(betas<0))[0]
    poscells = np.repeat(False, d.obs.shape[0])
    negcells = np.repeat(False, d.obs.shape[0])
    i_poscells = [i for i in np.arange(poscells.shape[0]) if d.obs['aparna2p0'].iloc[i] in posclust]
    i_negcells = [i for i in np.arange(negcells.shape[0]) if d.obs['aparna2p0'].iloc[i] in negclust]
    poscells[i_poscells] = True
    negcells[i_negcells] = True
    masc_globalp.append(res[0]) # Global p with correction for num clusters
    poscells_masc = np.concatenate((poscells_masc.reshape(i_pheno+1,-1), poscells.reshape(1,-1)), axis = 0)
    negcells_masc = np.concatenate((negcells_masc.reshape(i_pheno+1,-1), negcells.reshape(1,-1)), axis = 0)

    # Save Results
    f = open('masc_globalp.txt','wb')
    pickle.dump(masc_globalp, f)

    f = open('negcells_masc.txt','wb')
    pickle.dump(negcells_masc, f)

    f = open('poscells_masc.txt','wb')
    pickle.dump(poscells_masc, f)
    
    print(i_pheno)

In [ ]:
# Load Saved Results
f = open('masc_globalp.txt','rb')
masc_globalp = pickle.load(f)

f = open('negcells_masc.txt','rb')
negcells_masc = pickle.load(f)

f = open('poscells_masc.txt','rb')
poscells_masc = pickle.load(f)

poscells_masc = poscells_masc[np.arange(1,poscells_masc.shape[0]),:]
negcells_masc = negcells_masc[np.arange(1,negcells_masc.shape[0]),:]

In [ ]:
np.sum(np.array(masc_globalp)<0.05/17)

In [ ]:
QCed_phenos[np.where(np.array(masc_globalp)<0.05/len(QCed_phenos))[0]]

In [ ]:
masc_globalp = np.array(masc_globalp)

### Generate CNA Survey of Phenotypes

In [ ]:
np.random.seed(0) # for reproducibility
n_phenos = len(QCed_phenos)
global_ps = []
var_explained = []
newly_sig = []
nnull_inits = np.repeat(1000,len(QCed_phenos))
#i_replace = [i for i in np.arange(len(QCed_phenos)) if QCed_phenos[i] in ['EURad4KR']]
#nnull_inits[i_replace] = 100000
#i_replace = [i for i in np.arange(len(QCed_phenos)) if \
#             QCed_phenos[i] in ['Sex_M', 'age', 'season_Winter']]
#nnull_inits[i_replace] = 1000000
covs = d.samplem[['nUMI', 'percent_mito']].values 
for i_pheno in np.arange(len(QCed_phenos)):
    print(str(i_pheno)+":  "+QCed_phenos[i_pheno])
    sel_covars = selected_covars.iloc[i_pheno,:]
    cov_names = np.concatenate((sel_covars.index[sel_covars],['nUMI','percent_mito']))
    covs = d.samplem[cov_names].values
    y = d.samplem[QCed_phenos[i_pheno]].values
    res = cna.tl._association.association(d, y, batches=batches, covs=covs, Nnull=nnull_inits[i_pheno],
                                     local_test = False)
    global_ps.append(res.p)
    var_explained.append(res.r2)

global_ps = np.array(global_ps)
var_explained = np.array(var_explained)

In [ ]:
# Save Results
f = open('global_ps.txt','wb')
pickle.dump(global_ps, f)

f = open('var_explained.txt','wb')
pickle.dump(var_explained, f)

In [ ]:
QCed_phenos[global_ps<0.05]

In [ ]:
global_ps

In [ ]:
# Load Saved Results
f = open('global_ps.txt','rb')
global_ps= pickle.load(f)

f = open('var_explained.txt','rb')
var_explained= pickle.load(f)

In [ ]:
n_phenos = len(QCed_phenos)

In [ ]:
i_newly_sig = np.where((global_ps<0.05/n_phenos)*(masc_globalp>0.05/17))[0]

In [ ]:
newly_sig = np.repeat(False, len(QCed_phenos))
newly_sig[i_newly_sig]=True

In [ ]:
QCed_phenos[newly_sig]

In [ ]:
globalsig_phenos = QCed_phenos[np.where(np.array(global_ps)<0.05/n_phenos)[0]]
globalsig_phenos

In [ ]:
from adjustText import adjust_text

In [ ]:
# Where permutations maxed-out
QCed_phenos[np.where(global_ps==np.min(global_ps))[0]]

In [ ]:
n_phenos = len(QCed_phenos)
loc_newly_sig = np.where(newly_sig)[0]
loc_unchanged = np.where(~np.array(newly_sig))[0]
plt.scatter(var_explained[loc_newly_sig], -np.log10(global_ps[loc_newly_sig]),c = "green", s = 2)
plt.scatter(var_explained[loc_unchanged], -np.log10(global_ps[loc_unchanged]),c = "navy", s = 2)
plt.axhline(-np.log10(0.05/n_phenos), c = "black", linestyle = "dashed")
plt.annotate("Bonferroni Threshold", (0.3, -np.log10(0.05/n_phenos)+0.1))
texts = [plt.text(var_explained[i], -np.log10(global_ps[i]), 
                  QCed_phenos[i], ha='center', va='center', fontsize = 5) for i in np.arange(len(var_explained))]
adjust_text(texts)
plt.xlabel("Variance Explained")
plt.ylabel("-log10(P-value)")

In [ ]:
plt.scatter(-np.log10(masc_globalp), -np.log10(global_ps),c = "black", s = 2)
plt.axhline(-np.log10(0.05/n_phenos), c = "black", linestyle = "dashed")
plt.axvline(-np.log10(0.05/n_phenos), c = "black", linestyle = "dashed")
texts = [plt.text(-np.log10(masc_globalp[i]), -np.log10(global_ps[i]), 
                  QCed_phenos[i], ha='center', va='center', fontsize = 5) for i in np.arange(len(var_explained))]
adjust_text(texts)
plt.xlabel("-log10(P-value) MASC")
plt.ylabel("-log10(P-value) CNA")

### Local Association Testing for Globally-Significant Phenotypes

In [ ]:
# Age 
np.random.seed(0)
pheno = "age"
i_pheno = np.where(QCed_phenos==pheno)[0][0]
sel_covars = selected_covars.iloc[i_pheno,:]
cov_names = np.concatenate((sel_covars.index[sel_covars],['nUMI','percent_mito']))
covs = d.samplem[cov_names].values
y = d.samplem[QCed_phenos[i_pheno]].values
age_res = cna.tl._association.association(d, y,batches=batches, covs=covs, Nnull=1000)
age_res.keptcells = d.uns['keptcells']
f = open('age_res.txt','wb')
pickle.dump(age_res, f)

In [ ]:
# European ancestry
np.random.seed(0)
pheno = "EURad4KR"
i_pheno = np.where(QCed_phenos==pheno)[0][0]
sel_covars = selected_covars.iloc[i_pheno,:]
cov_names = np.concatenate((sel_covars.index[sel_covars],['nUMI','percent_mito']))
covs = d.samplem[cov_names].values
y = d.samplem[QCed_phenos[i_pheno]].values
ancestry_res = cna.tl._association.association(d, y, batches=batches, covs=covs, Nnull=1000)
ancestry_res.keptcells = d.uns['keptcells']
f = open('ancestry_res.txt','wb')
pickle.dump(ancestry_res, f)

In [ ]:
# Sex set phenotype and covariates to match previous analysis
np.random.seed(0)
pheno = "Sex_M"
i_pheno = np.where(QCed_phenos==pheno)[0][0]
sel_covars = selected_covars.iloc[i_pheno,:]
cov_names = np.concatenate((sel_covars.index[sel_covars],['nUMI','percent_mito']))
covs = d.samplem[cov_names].values
y = d.samplem[QCed_phenos[i_pheno]].values
sex_res = cna.tl._association.association(d, y, batches=batches, covs=covs, Nnull=1000)
sex_res.keptcells = d.uns['keptcells']
f = open('sex_res.txt','wb')
pickle.dump(sex_res, f)

In [ ]:
# Winter season set phenotype and covariates to match previous analysis
np.random.seed(0)
pheno = "season_Winter"
i_pheno = np.where(QCed_phenos==pheno)[0][0]
sel_covars = selected_covars.iloc[i_pheno,:]
cov_names = np.concatenate((sel_covars.index[sel_covars],['nUMI','percent_mito']))
covs = d.samplem[cov_names].values
y = d.samplem[QCed_phenos[i_pheno]].values
winter_res = cna.tl._association.association(d, y, batches=batches, covs=covs, Nnull=1000)
winter_res.keptcells = d.uns['keptcells']
f = open('winter_res.txt','wb')
pickle.dump(winter_res, f)

In [ ]:
# Load Saved Results
f = open('age_res.txt','rb')
age_res = pickle.load(f)

f = open('sex_res.txt','rb')
sex_res = pickle.load(f)

f = open('ancestry_res.txt','rb')
ancestry_res = pickle.load(f)

f = open('winter_res.txt','rb')
winter_res = pickle.load(f)

In [ ]:
pheno_names = np.array(['Age','Height', 'Weight', 'European Ancestry',
       'num_scar', 'BCG_scar', 'BCG_vaccine_1.0',
       'Sex', 'edu_cat_belowHighschool', 'sesgroup2_high',
       'sesgroup2_low', 'sesgroup2_medium', 'smk_status_Non-Smoker',
       'DRK_status_Non-Drinker',
       'Malnutrition', 'Spring', 'Winter'])

In [ ]:
CNA_poscells_tot = np.array([np.sum(age_res.ncorrs>age_res.fdr_5p_t),
                            np.sum(ancestry_res.ncorrs>ancestry_res.fdr_5p_t),
                             np.sum(sex_res.ncorrs>sex_res.fdr_5p_t),
                            np.sum(winter_res.ncorrs>winter_res.fdr_5p_t)])

CNA_negcells_tot = np.array([np.sum(age_res.ncorrs<-age_res.fdr_5p_t),
                            np.sum(ancestry_res.ncorrs<-ancestry_res.fdr_5p_t),
                             np.sum(sex_res.ncorrs<-sex_res.fdr_5p_t),
                            np.sum(winter_res.ncorrs<-winter_res.fdr_5p_t)])

In [ ]:
# Compute fraction novel and fraction replicated for each phenotype
def compare_populations(phenoname, phenolist, masc_res_pos, masc_res_neg, cna_res_obj, i_cna_cells):
    i_pheno = np.where(phenolist==phenoname)[0]
    masc_pos_pheno = masc_res_pos[i_pheno,:]
    masc_neg_pheno = masc_res_neg[i_pheno,:]
    
    FDR_thresh = cna_res_obj.fdr_5p_t
    if phenoname=="Weight":
        FDR_thresh = cna_res_obj.fdr_10p_t
    
    cna_poscells = np.repeat(False, masc_neg_pheno.shape[1])
    if np.sum(cna_res_obj.ncorrs>FDR_thresh)>0:
        i_pos = i_cna_cells[np.where(cna_res_obj.ncorrs>FDR_thresh)[0]]
        cna_poscells[i_pos]=True
    
    cna_negcells = np.repeat(False, masc_neg_pheno.shape[1])
    if np.sum(cna_res_obj.ncorrs<-FDR_thresh)>0:
        i_neg = i_cna_cells[np.where(cna_res_obj.ncorrs<-FDR_thresh)[0]]
        cna_negcells[i_neg]=True
    
    # Out of all cells found by MASC, what fraction did CNA pick up
    intersect = np.sum((masc_pos_pheno*cna_poscells)+(masc_neg_pheno*cna_negcells))
    frac_replicated = intersect/np.sum(masc_pos_pheno+masc_neg_pheno)
    
    # Out of all cells found by CNA, what fraction were not found by MASC
    frac_novel = intersect/np.sum(cna_poscells+cna_negcells)
    
    # Number of PCs in model
    npcs_used = cna_res_obj.k
    
    return (frac_replicated, frac_novel, npcs_used)

In [ ]:
# Compute overlap between populations found by CNA and by Clustering
overlap_metrics = np.array([0,0,0]).reshape(1,3)

new_metrics = compare_populations("age", QCed_phenos, poscells_masc, negcells_masc, 
                                  age_res, i_survey_cells)
overlap_metrics = np.concatenate((overlap_metrics, np.array(new_metrics).reshape(1,3)), axis = 0)

new_metrics = compare_populations("EURad4KR", QCed_phenos, poscells_masc, negcells_masc, 
                                  ancestry_res, i_survey_cells)
overlap_metrics = np.concatenate((overlap_metrics, np.array(new_metrics).reshape(1,3)), axis = 0)

new_metrics = compare_populations("Sex_M", QCed_phenos, poscells_masc, negcells_masc, 
                                  sex_res, i_survey_cells)
overlap_metrics = np.concatenate((overlap_metrics, np.array(new_metrics).reshape(1,3)), axis = 0)

new_metrics = compare_populations("season_Winter", QCed_phenos, poscells_masc, negcells_masc, 
                                  winter_res, i_survey_cells)
overlap_metrics = np.concatenate((overlap_metrics, np.array(new_metrics).reshape(1,3)), axis = 0)

overlap_metrics = overlap_metrics[np.arange(1,overlap_metrics.shape[0]),:]

In [ ]:
i_globalsig_phenos = [i for i in np.arange(len(QCed_phenos)) if QCed_phenos[i] in globalsig_phenos]

In [ ]:
var_explained_print = [np.around(var_explained[i]*100,0).astype(int).astype(str)+"%" for i in i_globalsig_phenos]

In [ ]:
CNA_sigglobalp_round = ['{:.1e}'.format(global_ps[i]) for i in i_globalsig_phenos]
MASC_sigglobalp_round = ['Not sig.' if newly_sig[i] else '{:.1e}'.format(masc_globalp[i]) for i in i_globalsig_phenos]

In [ ]:
masc_poscells_print = ['NA' if newly_sig[i] else np.sum(poscells_masc[i,:]) for i in i_globalsig_phenos]
masc_negcells_print = ['NA' if newly_sig[i] else np.sum(negcells_masc[i,:]) for i in i_globalsig_phenos]

In [ ]:
pct_replicated_print = ['NA' if masc_negcells_print[i]=='NA' else np.around(overlap_metrics[i,0]*100,0).astype(int)\
                        for i in np.arange(overlap_metrics.shape[0])]

In [ ]:
pct_novel_print = ['100' if masc_negcells_print[i]=='NA' else np.around(overlap_metrics[i,1]*100,0).astype(int)\
                        for i in np.arange(overlap_metrics.shape[0])]

In [ ]:
### Make Supplemental Table of Results
suptab_df = pd.DataFrame({"Phenotype": pheno_names[i_globalsig_phenos],
                         "CNA Global P": CNA_sigglobalp_round,
                         "Var. Explained": var_explained_print,
                          "MASC Global P": MASC_sigglobalp_round,
                         "MASC +Cells": masc_poscells_print,
                         "MASC -Cells": masc_negcells_print,
                         "CNA +Cells": CNA_poscells_tot,
                         "CNA -Cells": CNA_negcells_tot,
                         "% Replicated": pct_replicated_print,
                         "% Novel": pct_novel_print}) 

suptab = suptab_df.iloc[np.argsort(global_ps[i_globalsig_phenos]),:]
suptab = suptab.reset_index(drop = True)

In [ ]:
suptab

In [ ]:
# Average number of NAM PCs used
np.around(np.mean(overlap_metrics[:,2]),0).astype(int)

In [ ]:
# Average variance explained
np.around(np.mean(var_explained[i_globalsig_phenos]),2)*100

In [ ]:
# Export age results for interpretation
age_df = pd.DataFrame({'cluster':d.obs['aparna2p0'],
                         'keptcells': np.repeat(False,d.obs.shape[0]),
                         'ncorrs': np.repeat(0,d.obs.shape[0]),
                         'poscells': np.repeat(False,d.obs.shape[0]),
                         'negcells': np.repeat(False,d.obs.shape[0])})
i_cells = np.where(age_res.keptcells)[0]
age_df['keptcells'].iloc[i_cells] = True
age_df['ncorrs'].iloc[i_cells] = age_res.ncorrs
age_df['poscells'].iloc[i_cells] = age_res.ncorrs>age_res.fdr_5p_t
age_df['negcells'].iloc[i_cells] = age_res.ncorrs<-age_res.fdr_5p_t
age_df['UMAP1'] = d.obsm['X_umap'][:,0]
age_df['UMAP2'] = d.obsm['X_umap'][:,1]
age_df.to_csv("/data/srlab/lrumker/MCSC_Project/mcsc_scratch/tbru_cna_Age_res.txt")

In [ ]:
np.sum(age_df['negcells'])/np.sum(age_df['keptcells'])

In [ ]:
np.sum(age_df['poscells'])/np.sum(age_df['keptcells'])

In [ ]:
# Export winter season results for interpretation
winter_df = pd.DataFrame({'cluster':d.obs['aparna2p0'],
                         'keptcells': np.repeat(False,d.obs.shape[0]),
                         'ncorrs': np.repeat(0,d.obs.shape[0]),
                         'poscells': np.repeat(False,d.obs.shape[0]),
                         'negcells': np.repeat(False,d.obs.shape[0])})
i_cells = np.where(winter_res.keptcells)[0]
winter_df['keptcells'].iloc[i_cells] = True
winter_df['ncorrs'].iloc[i_cells] = winter_res.ncorrs
winter_df['poscells'].iloc[i_cells] = winter_res.ncorrs>winter_res.fdr_5p_t
winter_df['negcells'].iloc[i_cells] = winter_res.ncorrs<-winter_res.fdr_5p_t
winter_df['UMAP1'] = d.obsm['X_umap'][:,0]
winter_df['UMAP2'] = d.obsm['X_umap'][:,1]
winter_df.to_csv("/data/srlab/lrumker/MCSC_Project/mcsc_scratch/tbru_cna_Winter_res.txt")

In [ ]:
np.sum(winter_df['negcells'])/np.sum(winter_df['keptcells'])

In [ ]:
np.sum(winter_df['poscells'])/np.sum(winter_df['keptcells'])

### TB Phenotype Analysis

Original publication observed cluster C-12 (#14) depleted among cases (OR = 0.80, 95% CI: 0.73–0.87, p = 1.21 x 10-6), with an average frequency of 3.0% and 3.6% in cases and controls.

..and a weaker but statistically significant depletion of cluster C-20 (#24) in cases (OR = 0.76, 95% CI: 0.65–0.89, p = 5.95 x 10-4) with 1.1% and 1.2% of cells in cases and controls

In [ ]:
# Set covariates to match published cluster-based analysis
covs = d.samplem[['nUMI', 'percent_mito', 'age', 'Sex_M', 'season_Winter', 'EURad4KR', 
                  'age']].values; covs[:,-1] = covs[:,-1]**2

TB_res = cna.tl._association.association(d, y = d.samplem.TB_STATUS_CASE.values,
                                      batches=batches, covs=covs, Nnull=10000)
i_tb_cells = np.where(d.uns['keptcells'])[0]

In [ ]:
TB_df = pd.DataFrame({'cluster':d.obs['aparna2p0'],
                         'keptcells': np.repeat(False,d.obs.shape[0]),
                         'ncorrs': np.repeat(0,d.obs.shape[0]),
                         'poscells': np.repeat(False,d.obs.shape[0]),
                         'negcells': np.repeat(False,d.obs.shape[0])})
TB_df['keptcells'].iloc[i_tb_cells] = True
TB_df['ncorrs'].iloc[i_tb_cells] = TB_res.ncorrs
TB_df['poscells'].iloc[i_tb_cells] = TB_res.ncorrs>TB_res.fdr_5p_t
TB_df['negcells'].iloc[i_tb_cells] = TB_res.ncorrs<-TB_res.fdr_5p_t
TB_df['UMAP1'] = d.obsm['X_umap'][:,0]
TB_df['UMAP2'] = d.obsm['X_umap'][:,1]
TB_df.to_csv("/data/srlab/lrumker/MCSC_Project/mcsc_scratch/tbru_cna_TBres.txt")

In [ ]:
TB_pos_table = pd.DataFrame({"Cluster Name": ['C-23', 'C-29', 'C-28'],
    "Population Fraction": round(TB_df.loc[TB_df.TB_poscells]['cluster'].value_counts()/np.sum(TB_df.TB_poscells),2)[0:3],
                            "Annotation": ['CD4+ cytotoxic', 'CD8+ GZMH+', 'CD8+ GZMK']})
TB_pos_table

In [ ]:
from methods import methods

In [ ]:
d.obs['cytotoxic_clust'] = np.repeat("0", d.obs.shape[0])
loc_cytotoxic = [i for i in np.arange(d.obs.shape[0]) if d.obs['aparna2p0'].iloc[i] in [7,13,15]]
d.obs['cytotoxic_clust'].iloc[loc_cytotoxic] ="1"

In [ ]:
d.obs['c23'] = np.repeat("0", d.obs.shape[0])
d.obs['c23'].iloc[np.where(d.obs['aparna2p0']==7)[0]] ="1"

In [ ]:
d.obs['c28'] = np.repeat("0", d.obs.shape[0])
d.obs['c28'].iloc[np.where(d.obs['aparna2p0']==15)[0]] ="1"

In [ ]:
d.obs['c29'] = np.repeat("0", d.obs.shape[0])
d.obs['c29'].iloc[np.where(d.obs['aparna2p0']==13)[0]] ="1"

In [ ]:
d.obs['c12'] = np.repeat("0", d.obs.shape[0])
d.obs['c12'].iloc[np.where(d.obs['aparna2p0']==14)[0]] ="1"

In [ ]:
from importlib import reload

In [ ]:
reload(methods)

In [ ]:
# To verify an association using MASC
covs = d.samplem[['nUMI', 'percent_mito', 'age', 'Sex_M', 'season_Winter', 'EURad4KR', 
                  'age']].values; covs[:,-1] = covs[:,-1]**2
res = methods._MASC(d, d.samplem.TB_STATUS_CASE.values, batches, d.samplem.C.values, 
                    covs, # sample-level covariates                                                                                                                                                      
                        d.obs[['nUMI', 'percent_mito']].values, # Cell-level covariates
                        clustertype='cytotoxic_clust')

In [ ]:
# To verify an association using MASC
covs = d.samplem[['nUMI', 'percent_mito', 'age', 'Sex_M', 'season_Winter', 'EURad4KR', 
                  'age']].values; covs[:,-1] = covs[:,-1]**2
res = methods._MASC(d, d.samplem.TB_STATUS_CASE.values, batches, d.samplem.C.values, 
                    covs, # sample-level covariates                                                                                                                                                      
                        d.obs[['nUMI', 'percent_mito']].values, # Cell-level covariates
                        clustertype='c23')

In [ ]:
# To verify an association using MASC
covs = d.samplem[['nUMI', 'percent_mito', 'age', 'Sex_M', 'season_Winter', 'EURad4KR', 
                  'age']].values; covs[:,-1] = covs[:,-1]**2
res = methods._MASC(d, d.samplem.TB_STATUS_CASE.values, batches, d.samplem.C.values, 
                    covs, # sample-level covariates                                                                                                                                                      
                        d.obs[['nUMI', 'percent_mito']].values, # Cell-level covariates
                        clustertype='c29')

In [ ]:
# To verify an association using MASC
covs = d.samplem[['nUMI', 'percent_mito', 'age', 'Sex_M', 'season_Winter', 'EURad4KR', 
                  'age']].values; covs[:,-1] = covs[:,-1]**2
res = methods._MASC(d, d.samplem.TB_STATUS_CASE.values, batches, d.samplem.C.values, 
                    covs, # sample-level covariates                                                                                                                                                      
                        d.obs[['nUMI', 'percent_mito']].values, # Cell-level covariates
                        clustertype='c28')

In [ ]:
from matplotlib.cm import get_cmap
colors = get_cmap("tab20b").colors

In [ ]:
# Plotting clusters
f, axs = plt.subplots(1, 1, figsize=(3,3))

ax = axs[1,1]
ax.axis('off')

i_survey_cells = np.where(age_res.keptcells)[0]
ax.scatter(d.obsm['X_umap'][i_survey_cells,0], d.obsm['X_umap'][i_survey_cells,1], 
           alpha=0.6, c="grey", cmap='seismic', **pp.umapprops)
i_sig_cells = np.where(np.abs(age_res.ncorrs)>age_res.fdr_5p_t)[0]
ax.scatter(d.obsm['X_umap'][i_survey_cells[i_sig_cells],0], 
           d.obsm['X_umap'][i_survey_cells[i_sig_cells],1], 
           alpha=0.6, c=c[i_sig_cells], cmap='seismic', 
           vmin=-cutoff, vmax=cutoff, **pp.umapprops)
ax.set_title("Age: CNA Populations")

In [ ]:
f, axs = plt.subplots(3, 3, figsize=(6,6))

ax = axs[2,0]
loc_newly_sig = np.where(newly_sig)[0]
loc_unchanged = np.where(~np.array(newly_sig))[0]
ax.scatter(var_explained[loc_newly_sig], -np.log10(global_ps[loc_newly_sig]),c = "lime", s = 2)
ax.scatter(var_explained[loc_unchanged], -np.log10(global_ps[loc_unchanged]),c = colors[0], s = 2)
ax.axhline(-np.log10(0.05/n_phenos), c = "black", linestyle = "dashed")
ax.annotate("Bonferroni", (0.3, -np.log10(0.05/n_phenos)+0.1), fontsize = 5)
loc_sig = np.where(global_ps <0.05/n_phenos)[0]
texts = [ax.text(var_explained[i], -np.log10(global_ps[i]), 
                  pheno_names[i], ha='center', va='center', fontsize = 5) for i in loc_sig]
adjust_text(texts, arrowprops=dict(arrowstyle="-", color='k', lw=0.5), ax=ax)
ax.set_xlabel("Variance Explained")
ax.set_ylabel("-log10(P-value)")

ax = axs[1,0]
ax.axis('off')
c = ancestry_res.ncorrs
c = (c-np.mean(c))/np.std(c)
cutoff = np.max([-np.percentile(c, 10), np.percentile(c, 90)]) 
i_survey_cells = np.where(ancestry_res.keptcells)[0]
ax.scatter(d.obsm['X_umap'][i_survey_cells,0], d.obsm['X_umap'][i_survey_cells,1], 
           alpha=0.6, c="grey", cmap='seismic', **pp.umapprops)
i_sig_cells = np.where(np.abs(ancestry_res.ncorrs)>ancestry_res.fdr_5p_t)[0]
ax.scatter(d.obsm['X_umap'][i_survey_cells[i_sig_cells],0], 
           d.obsm['X_umap'][i_survey_cells[i_sig_cells],1], 
           alpha=0.6, c=c[i_sig_cells], cmap='seismic', 
           vmin=-cutoff, vmax=cutoff, **pp.umapprops)
ax.set_title("Ancestry: CNA Populations")

ax = axs[1,1]
ax.axis('off')
c = age_res.ncorrs
c = (c-np.mean(c))/np.std(c)
cutoff = np.max([-np.percentile(c, 10), np.percentile(c, 90)]) 
i_survey_cells = np.where(age_res.keptcells)[0]
ax.scatter(d.obsm['X_umap'][i_survey_cells,0], d.obsm['X_umap'][i_survey_cells,1], 
           alpha=0.6, c="grey", cmap='seismic', **pp.umapprops)
i_sig_cells = np.where(np.abs(age_res.ncorrs)>age_res.fdr_5p_t)[0]
ax.scatter(d.obsm['X_umap'][i_survey_cells[i_sig_cells],0], 
           d.obsm['X_umap'][i_survey_cells[i_sig_cells],1], 
           alpha=0.6, c=c[i_sig_cells], cmap='seismic', 
           vmin=-cutoff, vmax=cutoff, **pp.umapprops)
ax.set_title("Age: CNA Populations")

ax = axs[2,1]
ax.axis('off')
pheno = "age"
i_pheno = np.where(QCed_phenos==pheno)[0]
pos_cells = np.where(poscells_masc[i_pheno,:][0])[0]
neg_cells = np.where(negcells_masc[i_pheno,:][0])[0]
c = np.repeat("grey", d.obs.shape[0])
c[pos_cells] = "red"
c[neg_cells] = "blue"
ax.scatter(d.obsm['X_umap'][i_survey_cells,0], d.obsm['X_umap'][i_survey_cells,1], 
           alpha=0.6, c=c[i_survey_cells], cmap='seismic', 
           vmin=-cutoff, vmax=cutoff, **pp.umapprops)
ax.set_title("Age: Clusters")

ax = axs[1,2]
ax.axis('off')
c = winter_res.ncorrs
c = (c-np.mean(c))/np.std(c)
cutoff = np.max([-np.percentile(c, 10), np.percentile(c, 90)]) 
i_survey_cells = np.where(winter_res.keptcells)[0]
ax.scatter(d.obsm['X_umap'][i_survey_cells,0], d.obsm['X_umap'][i_survey_cells,1], 
           alpha=0.6, c="grey", cmap='seismic', **pp.umapprops)
i_sig_cells = np.where(np.abs(winter_res.ncorrs)>winter_res.fdr_5p_t)[0]
ax.scatter(d.obsm['X_umap'][i_survey_cells[i_sig_cells],0], 
           d.obsm['X_umap'][i_survey_cells[i_sig_cells],1], 
           alpha=0.6, c=c[i_sig_cells], cmap='seismic', 
           vmin=-cutoff, vmax=cutoff, **pp.umapprops)
ax.set_title("Winter: CNA Populations")

ax = axs[2,2]
ax.axis('off')
pheno = "season_Winter"
i_pheno = np.where(QCed_phenos==pheno)[0]
pos_cells = np.where(poscells_masc[i_pheno,:][0])[0]
neg_cells = np.where(negcells_masc[i_pheno,:][0])[0]
c = np.repeat("grey", d.obs.shape[0])
c[pos_cells] = "red"
c[neg_cells] = "blue"
ax.scatter(d.obsm['X_umap'][i_survey_cells,0], d.obsm['X_umap'][i_survey_cells,1], 
           alpha=0.6, c=c[i_survey_cells], cmap='seismic', 
           vmin=-cutoff, vmax=cutoff, **pp.umapprops)
ax.set_title("Winter: Clusters")

ax = axs[0,0]
ax.axis('off')
c = TB_res.ncorrs
c = (c-np.mean(c))/np.std(c)
cutoff = np.max([-np.percentile(c, 10), np.percentile(c, 90)]) 
ax.scatter(d.obsm['X_umap'][i_tb_cells,0], d.obsm['X_umap'][i_tb_cells,1], 
           alpha=0.6, c="grey", cmap='seismic', **pp.umapprops)
i_sig_cells = np.where(np.abs(TB_res.ncorrs)>TB_res.fdr_5p_t)[0]
ax.scatter(d.obsm['X_umap'][i_tb_cells[i_sig_cells],0], 
           d.obsm['X_umap'][i_tb_cells[i_sig_cells],1], 
           alpha=0.6, c=c[i_sig_cells], cmap='seismic', 
           vmin=-cutoff, vmax=cutoff, **pp.umapprops)
ax.set_title("TB: CNA Populations")

ax = axs[0,1]
ax.axis('off')
ax.scatter(d.obsm['X_umap'][i_tb_cells,0], d.obsm['X_umap'][i_tb_cells,1], 
           alpha=0.6, c="grey", cmap='seismic', **pp.umapprops)
i_sig_cells = np.where(np.abs(TB_res.ncorrs)>TB_res.fdr_5p_t)[0]

plot_df = pd.DataFrame({"UMAP1": d.obsm['X_umap'][i_tb_cells[i_sig_cells],0],
                       "UMAP2": d.obsm['X_umap'][i_tb_cells[i_sig_cells],1],
                       "Cluster": d.obs['aparna2p0'][i_tb_cells[i_sig_cells]]})
plot_df = plot_df.iloc[np.random.choice(np.arange(plot_df.shape[0]),plot_df.shape[0], replace=False),:]
counts = plot_df.Cluster.value_counts()
keep_clusts = counts.index[counts.values>100] # Only plot cells from clusters with at least 100 cells implicated
i_keep_cells = [i for i in np.arange(plot_df.shape[0]) if plot_df.Cluster[i] in keep_clusts]
plot_df = plot_df.iloc[i_keep_cells,:]
ax.scatter(plot_df['UMAP1'], plot_df['UMAP2'],
           alpha=0.6, c=plot_df['Cluster'], cmap = 'tab20b', **pp.umapprops)
ax.set_title("TB: CNA Populations,\nColored by Cluster")

ax = axs[0,2]
ax.axis('off')
ax.scatter(d.obsm['X_umap'][i_tb_cells,0], d.obsm['X_umap'][i_tb_cells,1], 
           alpha=0.6, c="grey", cmap='seismic', **pp.umapprops)
C12_cells = [i for i in np.arange(d.obs.shape[0]) if d.obs['aparna2p0'][i]==14] #C12 corresponds to cluster 14
C20_cells = [i for i in np.arange(d.obs.shape[0]) if d.obs['aparna2p0'][i]==24] #C20 corresponds to cluster 24
c = np.repeat("grey", d.obs.shape[0])
ax.scatter(d.obsm['X_umap'][C12_cells,0], d.obsm['X_umap'][C12_cells,1], 
           alpha=0.6, c=colors[11], **pp.umapprops)
ax.scatter(d.obsm['X_umap'][C20_cells,0], d.obsm['X_umap'][C20_cells,1], 
           alpha=0.6, c=colors[19], **pp.umapprops)
ax.set_title("TB:  Clusters")

plt.tight_layout(h_pad=1.0)
plt.savefig('../_figs/rawmainfig.tbru.supervised.pdf')

# Examine TB Associations in Depth

### What fraction of C12 and C20 cells are included among our negatively-associated population?
Note that publication renamed clusters in order of innateness, so that C12 corresponds to cluster #14 and C20 to #24

In [ ]:
np.sum((d.obs['aparna2p0']==14)*TB_df['negcells'])/np.sum(d.obs['aparna2p0']==14)*100

In [ ]:
np.sum((d.obs['aparna2p0']==24)*TB_df['negcells'])/np.sum(d.obs['aparna2p0']==24)*100

### Number of cells in CNA's negatively-associated population that are not in C12 or C20?

In [ ]:
n_remainder_negcells = np.sum(TB_df['negcells']) - \
                    np.sum((d.obs['aparna2p0']==14)*TB_df['negcells'])-\
                    np.sum((d.obs['aparna2p0']==24)*TB_df['negcells'])
n_remainder_negcells

### What percent of CNA's negatively-associated population fall outside C12 and C20?

In [ ]:
n_remainder_negcells/np.sum(TB_df['negcells'])*100

### For the cells in CNA's negatively-associated population that fell outside of C12, what clusters did they belong to?

 - Cluster #0  = C4 = "CD4+ CD27+ Th17"
 - Cluster #9 = C16 = "CD4+ CCR6+CXCR3+ Th1/Th17"
 - Cluster #11 = C19 = "CD4+ CD161+ Th1"
 - Cluster #2 = C17 = "CD4+ CD29+CXCR3+ Th1"

In [ ]:
loc_remainder_negcells = np.where((d.obs['aparna2p0']!=14)*(d.obs['negcells']=='1'))[0]
d.obs['aparna2p0'][loc_remainder_negcells].value_counts()

### Gene set enrichment

In [ ]:
d.obs['keptcells'] = d.uns['keptcells']

In [ ]:
d.obs['ncorrs'] = np.repeat(0, d.obs.shape[0])

In [ ]:
d.obs['ncorrs'].loc['keptcells'] = res.ncorrs

In [ ]:
dummy_df = pd.DataFrame(d.obs.loc[:,["ncorrs", "poscells", "negcells", "aparna2p0", 'keptcells']])
dummy_df.to_csv("/data/srlab/lrumker/MCSC_Project/mcsc_scratch/tbru_cna_res.txt")

In [ ]:
dummy_df = pd.DataFrame(d.uns['NAM_nbhdXpc'].iloc[:,0:10])
dummy_df.to_csv("/data/srlab/lrumker/MCSC_Project/mcsc_scratch/tbru_cna_NAM_PCs.txt")

In [ ]:
### Read in data types

print('reading')
d_cca = cna.read(pf.tbru_h5ad + 'harmcca20.h5ad')
d_cca.obs_to_sample(['batch', 'nUMI', 'percent_mito'])

print('reading')
d_mrna = cna.read(pf.tbru_h5ad + 'harmmrna.h5ad')
d_mrna.obs_to_sample(['batch', 'nUMI', 'percent_mito'])

print('reading')
d_prot = cna.read(pf.tbru_h5ad + 'harmprot.h5ad')
d_prot.obs_to_sample(['batch', 'nUMI', 'percent_mito'])

### Run analyses of TB progressor phenotype with each dataset

# Set covariates to match published cluster-based analysis
batches = d_cca.samplem.batch.values
covs = d_cca.samplem[['nUMI', 'percent_mito', 'age', 'Sex_M', 'season_Winter', 'EURad4KR', 
                  'age']].values; covs[:,-1] = covs[:,-1]**2

res_cca = cna.tl._association.association(d_cca, y = d_cca.samplem.TB_STATUS_CASE.values,
                                      batches=batches, covs=covs, Nnull=10000)

# Set covariates to match published cluster-based analysis
batches = d_mrna.samplem.batch.values
covs = d_mrna.samplem[['nUMI', 'percent_mito', 'age', 'Sex_M', 'season_Winter', 'EURad4KR', 
                  'age']].values; covs[:,-1] = covs[:,-1]**2

res_mrna = cna.tl._association.association(d_mrna, y = d_mrna.samplem.TB_STATUS_CASE.values,
                                      batches=batches, covs=covs, Nnull=10000)

# Set covariates to match published cluster-based analysis
batches = d_prot.samplem.batch.values
covs = d_prot.samplem[['nUMI', 'percent_mito', 'age', 'Sex_M', 'season_Winter', 'EURad4KR', 
                  'age']].values; covs[:,-1] = covs[:,-1]**2

res_prot = cna.tl._association.association(d_prot, y = d_prot.samplem.TB_STATUS_CASE.values,
                                      batches=batches, covs=covs, Nnull=10000)

### Compare results from different data types
print(res_cca.p)
print(res_mrna.p)
print(res_prot.p)

### Number of cells passing FDR 5%
print(np.sum(np.abs(res_cca.ncorrs)>res_cca.fdr_5p_t))
print(np.sum(np.abs(res_mrna.ncorrs)>res_mrna.fdr_5p_t))
print(np.sum(np.abs(res_prot.ncorrs)>res_prot.fdr_5p_t))

In [ ]:
### Identify overlap in implicated cell populations
# Locate associated cells CCA

cca_poscells = np.repeat(False, d_cca.obs.shape[0])
i_pos = np.where(d_cca.uns['keptcells'])[0][np.where(res_cca.ncorrs>res_cca.fdr_5p_t)[0]]
cca_poscells[i_pos]=True

cca_negcells = np.repeat(False, d_cca.obs.shape[0])
i_neg = np.where(d_cca.uns['keptcells'])[0][np.where(res_cca.ncorrs<-res_cca.fdr_5p_t)[0]]
cca_negcells[i_neg]=True

# Locate associated cells mRNA

mrna_poscells = np.repeat(False, d_mrna.obs.shape[0])
i_pos = np.where(d_mrna.uns['keptcells'])[0][np.where(res_mrna.ncorrs>res_mrna.fdr_5p_t)[0]]
mrna_poscells[i_pos]=True

mrna_negcells = np.repeat(False, d_mrna.obs.shape[0])
i_neg = np.where(d_mrna.uns['keptcells'])[0][np.where(res_mrna.ncorrs<-res_mrna.fdr_5p_t)[0]]
mrna_negcells[i_neg]=True

# Locate associated cells protein

prot_poscells = np.repeat(False, d_prot.obs.shape[0])
i_pos = np.where(d_prot.uns['keptcells'])[0][np.where(res_prot.ncorrs>res_prot.fdr_5p_t)[0]]
prot_poscells[i_pos]=True

prot_negcells = np.repeat(False, d_prot.obs.shape[0])
i_neg = np.where(d_prot.uns['keptcells'])[0][np.where(res_prot.ncorrs<-res_prot.fdr_5p_t)[0]]
prot_negcells[i_neg]=True

datatypes = np.array(['cca', 'mrna', 'prot'])
all_poscells = np.concatenate((np.concatenate((cca_poscells.reshape(1,-1), mrna_poscells.reshape(1,-1)),axis = 0),
                              prot_poscells.reshape(1,-1)),axis = 0)
all_negcells = np.concatenate((np.concatenate((cca_negcells.reshape(1,-1), mrna_negcells.reshape(1,-1)),axis = 0),
                              prot_negcells.reshape(1,-1)),axis = 0)

# Overlapping cells among data types
harvest_pos = np.zeros((3,3))
harvest_neg = np.zeros((3,3))
for i_type in np.arange(len(datatypes)):
    for j_type in np.arange(len(datatypes)):
        harvest_pos[i_type, j_type] = np.sum(all_poscells[i_type,:]*all_poscells[j_type,:])
        harvest_neg[i_type, j_type] = np.sum(all_negcells[i_type,:]*all_negcells[j_type,:])

In [ ]:
mrna_poscells[0:5]

In [ ]:
# Fraction of mRNA cells in C-12 or C-20
np.sum(mrna_negcells*((d.obs['aparna2p0']==14)+(d.obs['aparna2p0']==24)))/np.sum(mrna_negcells)

In [ ]:
# Fraction of mRNA cells in CCA depleted cluster
np.sum(mrna_negcells*cca_negcells)/np.sum(mrna_negcells)

### Verify an Assoc. With MASC

In [ ]:
d.obs['aparna2p0_elimsoloclust'] = d.obs['aparna2p0_elimsoloclust'].astype(str)

In [ ]:
d.obs['aparna2p0_elimsoloclust'] = d.obs['aparna2p0'].copy()
clust_sizes = d.obs['aparna2p0_elimsoloclust'].value_counts()
singleton_clusts = d.obs['aparna2p0_elimsoloclust'].value_counts().index[np.where(clust_sizes==1)[0]]
singleton_clust_locs = [i for i in np.arange(d.obs.shape[0]) if d.obs['aparna2p0_elimsoloclust'][i] in singleton_clusts]
d.obs.iloc[singleton_clust_locs,np.where(d.obs.columns=='aparna2p0_elimsoloclust')[0]] = 32

In [ ]:
sampleXmeta=sm

In [ ]:
# To verify an association using MASC
res = methods._MASC(d,
      y,
      sampleXmeta.batch.values,
      sampleXmeta.C.values,
      covs, # sample-level covariates                                                                                                                                                      
        d.obs[['nUMI', 'percent_mito']].values, # Cell-level covariates
      clustertype='poscells')

In [ ]:
# Formula to correct a MASC association p-value for total number of clusters tested
# ("Sidak Correction")
num_clusters = 31
uncorrected_pval = 0.000002
1-((1-uncorrected_pval)**num_clusters)

# Examine Winter Association In-Depth

In [ ]:
import anndata as ad

In [ ]:
pos_cells_winter = ad.AnnData(d.X[np.where(winter_df.keptcells&winter_df.poscells)[0],])
sc.pp.neighbors(pos_cells_winter)
sc.tl.umap(pos_cells_winter)

In [ ]:
loc_poscells_winter = np.where(winter_df.keptcells&winter_df.poscells)[0]

In [ ]:
data_df['Category'].value_counts()

In [ ]:
# Color cells by cluster, in original dataset's UMAP
data_df = pd.DataFrame({"X Value": d.obsm['X_umap'][loc_poscells_winter,0],
                     "Y Value": d.obsm['X_umap'][loc_poscells_winter,1],
                     "Category": d.obs['aparna2p0'][loc_poscells_winter]})
cluster_rep = data_df['Category'].value_counts()
keep_clust = cluster_rep.index[cluster_rep.values>10]
keep_row = [data_df['Category'][i] in keep_clust for i in np.arange(data_df.shape[0])]
data_df = data_df.iloc[keep_row,:]
data_df = data_df.sort_values("Category", ascending=False)
groups = data_df.groupby("Category", sort=False)

fig, ax = plt.subplots(figsize=(10, 6))
ax.margins(0.05) 
palette = plt.get_cmap('tab20').colors
cc = 0 #color counter
for name, group in groups:
    x_vals = group["X Value"]
    y_vals = group["Y Value"]
    plt.scatter(x_vals, y_vals, marker="o", label=name, s=3, alpha = 0.6,
               c=np.array(palette[cc]).reshape(1,-1)) #Used for legend and dots
    cc = cc+1
plt.legend(bbox_to_anchor=(0.85,0.5), loc="center right", fontsize=12, 
           bbox_transform=plt.gcf().transFigure, markerscale=2)
plt.subplots_adjust(left=0.0, bottom=0.1, right=0.6)
ax.set_xlabel('Original UMAP1', fontsize = 15)
ax.set_ylabel('Original UMAP2', fontsize = 15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("Expanded Population, \n Colored by Nathan et al Clusters",
         fontsize = 18)
plt.grid(color='lightgrey', linestyle='-', linewidth=0.5)
plt.show()

In [ ]:
# Color top 10% cells by cell-type cluster, after independently UMAPped
data_df = pd.DataFrame({"X Value": pos_cells_winter.obsm['X_umap'][:,0],
                     "Y Value": pos_cells_winter.obsm['X_umap'][:,1],
                     "Category": d.obs['aparna2p0'][loc_poscells_winter]})
cluster_rep = data_df['Category'].value_counts()
keep_clust = cluster_rep.index[cluster_rep.values>10]
keep_row = [data_df['Category'][i] in keep_clust for i in np.arange(data_df.shape[0])]
data_df = data_df.iloc[keep_row,:]
data_df = data_df.sort_values("Category", ascending=False)
groups = data_df.groupby("Category", sort=False)

fig, ax = plt.subplots(figsize=(10, 6))
ax.margins(0.05) 
palette = plt.get_cmap('tab20').colors
cc = 0 #color counter
for name, group in groups:
    x_vals = group["X Value"]
    y_vals = group["Y Value"]
    plt.scatter(x_vals, y_vals, marker="o", label=name, s=5, alpha = 0.6,
               c=np.array(palette[cc]).reshape(1,-1)) #Used for legend and dots
    cc = cc+1
plt.legend(bbox_to_anchor=(0.85,0.5), loc="center right", fontsize=12, 
           bbox_transform=plt.gcf().transFigure, markerscale=2)
plt.subplots_adjust(left=0.0, bottom=0.1, right=0.6)
ax.set_xlabel('New UMAP1', fontsize = 15)
ax.set_ylabel('New UMAP2', fontsize = 15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("Expanded Population, \n Colored by Nathan et al Clusters",
         fontsize = 18)
plt.grid(color='lightgrey', linestyle='-', linewidth=0.5)
plt.show()

In [ ]:
sc.tl.leiden(pos_assoc_data, resolution=0.2)

In [ ]:
# Color top 10% cells by cell-type cluster, after independently UMAPped and clustered
data_df = pd.DataFrame({"X Value": pos_assoc_data.obsm['X_umap'][:,0],
                     "Y Value": pos_assoc_data.obsm['X_umap'][:,1],
                     "Category": pos_assoc_data.obs['leiden']})
data_df = data_df.sort_values("Category", ascending=False)
groups = data_df.groupby("Category", sort=False)

fig, ax = plt.subplots(figsize=(10, 6))
ax.margins(0.05) 
palette = np.array(sns.color_palette("husl", len(np.unique(data_df['Category']))))
cc = 0 #color counter
for name, group in groups:
    x_vals = group["X Value"]
    y_vals = group["Y Value"]
    plt.scatter(x_vals, y_vals, marker="o", label=name, s=5, alpha = 0.6,
               c=palette[cc:cc+1,:]) #Used for legend and dots
    cc = cc+1
plt.legend(bbox_to_anchor=(0.85,0.5), loc="center right", fontsize=12, 
           bbox_transform=plt.gcf().transFigure, markerscale=2)
plt.subplots_adjust(left=0.0, bottom=0.1, right=0.6)
ax.set_xlabel('UMAP1', fontsize = 15)
ax.set_ylabel('UMAP2', fontsize = 15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("Cells Most Positively Assoc. with Phenotype",
         fontsize = 18)
plt.grid(color='lightgrey', linestyle='-', linewidth=0.5)
plt.show()

new_clust = 0
compare_clusterings = pd.DataFrame({"Cluster "+str(new_clust): data.obs['leiden2'][pos_assoc_cells][np.where(pos_assoc_data.obs['leiden']==str(new_clust))[0]].value_counts()})
for new_clust in np.arange(1, np.max(pos_assoc_data.obs['leiden'].astype(int))+1):
    compare_clusterings["Cluster "+str(new_clust)] = data.obs['leiden2'][pos_assoc_cells][np.where(pos_assoc_data.obs['leiden']==str(new_clust))[0]].value_counts()
    
compare_clusterings_pct = compare_clusterings/np.sum(compare_clusterings, axis = 0)

counts = compare_clusterings_pct.transpose()
fig = plt.figure(figsize=(13,7))
ax = fig.add_subplot(111)
counts.plot(ax=ax,kind='bar', stacked=True, rot=0)
vals = ax.get_yticks()
ax.set_yticklabels(['{:3.2f}%'.format(x*100) for x in vals])
ax.yaxis.grid(True)
ax.set_axisbelow(True)
ax.set_xlabel('Clusters Among Top-Scoring Cells', fontsize = 20)
ax.set_ylabel('Fraction of Cells from Each Original Cluster', fontsize = 20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.title("Cells Most Positively Assoc. with Phenotype",
         fontsize = 30)
plt.show()

# Interpretation: Exploring

In [ ]:
FDR_thresh = res.fdr_5p_t

In [ ]:
# Cell scores per neighborhood
d.obs['ncorrs'] = np.repeat(np.nan, d.obs.shape[0])
d.obs.loc[d.uns['keptcells'],d.obs.columns=='ncorrs'] = res.ncorrs

# Positively-associated cells
d.obs['poscells'] = np.repeat(False, d.obs.shape[0])
loc_pos_cells = np.arange(d.obs.shape[0])[np.where(d.obs['ncorrs']>FDR_thresh)[0]]
d.obs.iloc[loc_pos_cells, np.where(d.obs.columns=='poscells')[0]] = True

# Negatively-associated cells
d.obs['negcells'] = np.repeat(False, d.obs.shape[0])
loc_neg_cells = np.arange(d.obs.shape[0])[np.where(d.obs['ncorrs']<-FDR_thresh)[0]]
d.obs.iloc[loc_neg_cells, np.where(d.obs.columns=='negcells')[0]] = True

In [ ]:
from scipy.stats import gaussian_kde

In [ ]:
d.obs['aparna2p0'].loc[d.uns['keptcells']].loc[d.obs['negcells'].loc[d.uns['keptcells']]].value_counts()

In [ ]:
d.obs['aparna2p0'].loc[d.uns['keptcells']].loc[d.obs['poscells'].loc[d.uns['keptcells']]].value_counts()

In [ ]:
# Density of cells associated with phenotype

# Plot all cells as backdrop
plt.scatter(d.obsm['X_umap'][:,0], d.obsm['X_umap'][:,1], c = "grey", 
       s = 4, alpha = 0.3, cmap = 'cividis')

# Density of most positively-associated cells
xy = d.obsm['X_umap'][d.obs['poscells'],:].T
z = gaussian_kde(xy)(xy)
plt.scatter(d.obsm['X_umap'][:,0][d.obs['poscells']], d.obsm['X_umap'][:,1][d.obs['poscells']], 
           c=z, s = 5, alpha = 0.6, edgecolor='', cmap= "Reds")

# Density of most negatively-associated cells
xy = d.obsm['X_umap'][d.obs['negcells'],:].T
z = gaussian_kde(xy)(xy)
plt.scatter(d.obsm['X_umap'][:,0][d.obs['negcells']], d.obsm['X_umap'][:,1][d.obs['negcells']],
           c=z, s = 5, alpha = 0.6, edgecolor='', cmap = "Blues")


plt.title("Density of Cells Associated with Phenotype", fontsize=20)
plt.savefig('/data/srlab/lrumker/MCSC_Project/mcsc_scratch/tbru_prot.pdf')

### Scatterplot of All Per-Neighborhood Scores (Note:  Suboptimal vis. d/t overplotting)

In [ ]:
plt.scatter(*d.obsm['X_umap'][res.kept].T, c = CNA_cellscores, s = 5, alpha = 0.1,
           cmap = "viridis")
plt.colorbar()

### Cell Density, Weighted by Score (Helps with overplotting)

In [ ]:
# Density plot by phenotype score
x = d.obsm['X_umap'][res.kept][:,0]
y = d.obsm['X_umap'][res.kept][:,1]

heatmap, xedges, yedges = np.histogram2d(x, y, bins=100, weights = CNA_cellscores) #weights
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

plt.clf()
plt.imshow(heatmap.T, extent=extent, origin='lower', cmap = "viridis")
plt.title("Density, Weighted by Phenotype Score", fontsize=20)
plt.show()

### Distribution of per-cell scores in each cannoical cluster

Helps screen for interesting heterogeneity!

In [ ]:
# Draw the density plot
cell_scores = CNA_cellscores
for cluster_sel in np.unique(d.obs['aparna2p0_elimsoloclust']):
    plt.figure()
    sns.distplot(cell_scores[np.where(d.obs['aparna2p0_elimsoloclust'][res.kept]==str(cluster_sel))[0]], hist = False, kde = True,
            kde_kws = {'shade': True, 'linewidth': 3},
             label = cluster_sel, color = "g")

### Like above, but now compares in-cluster to non-cluster distributions of per-nh scores

In [ ]:
d.obs['aparna2p0_elimsoloclust'] = [str(d.obs['aparna2p0_elimsoloclust'][i]) for i in np.arange(d.obs.shape[0])] 

In [ ]:
cell_phen_assoc = CNA_cellscores
for cluster_sel in np.unique(d.obs['aparna2p0_elimsoloclust'][res.kept]):
    d1 = {"Cell_Score": np.concatenate((cell_phen_assoc[np.where(d.obs['aparna2p0_elimsoloclust'][res.kept].values!=str(cluster_sel))[0]], 
                         cell_phen_assoc[np.where(d.obs['aparna2p0_elimsoloclust'][res.kept].values==str(cluster_sel))[0]]),
                        axis = 0),
        "Cell_Group": np.concatenate((np.repeat("Non-Cluster", np.sum(d.obs['aparna2p0_elimsoloclust'][res.kept].values!=str(cluster_sel))), 
                         np.repeat("Cluster "+str(cluster_sel), np.sum(d.obs['aparna2p0_elimsoloclust'][res.kept].values==str(cluster_sel)))),
                        axis = 0)}
    df = pd.DataFrame(data = d1)
    
    plt.figure()
    cell_groups = ['Non-Cluster', 'Cluster '+cluster_sel]

    plt.figure(figsize=(15,7))
    # Iterate through the groups
    cell_group = 'Non-Cluster'
    # Subset to the group
    subset = df[df['Cell_Group'] == cell_group]

    # Draw the density plot
    sns.distplot(subset['Cell_Score'], hist = False, kde = True,
                kde_kws = {'shade': True, 'linewidth': 3},
                 label = cell_group, color = "g")

    # Iterate through the groups
    cell_group = 'Cluster '+cluster_sel
    # Subset to the group
    subset = df[df['Cell_Group'] == cell_group]

    # Draw the density plot
    sns.distplot(subset['Cell_Score'], hist = False, kde = True,
                kde_kws = {'shade': True, 'linewidth': 3},
                 label = cell_group, color = "b")

    # Plot formatting
    plt.legend(prop={'size': 20})
    plt.title('')
    plt.xlabel('')
    plt.ylabel('')

In [ ]:
# What do our NAM PCs even capture?
for num_PC in np.arange(20):
    plt.figure(figsize=(8,6))
    plt.scatter(*d.obsm['X_umap'][res.kept, :].T, c = d.uns['NAMsvdV'][:,num_PC], s = 5, alpha = 0.1,
           cmap = "viridis")
    plt.title("NAM PC"+str(num_PC), fontsize=20)
    plt.colorbar()

In [ ]:
# Most negatively associated with phenotype
cell_phen_assoc = CNA_cellscores
threshold = res.fdr_5p_t
plt.figure(figsize=(10,7))
neg_assoc_cells = np.where(cell_phen_assoc<-threshold)[0]
plt.scatter(*d.obsm['X_umap'][res.kept, :].T, c = cell_phen_assoc<-threshold, s = 4, alpha = 0.3,
       cmap = 'cividis')
plt.title("Cells Negatively Associated with Phenotype", fontsize=20)
plt.colorbar()

In [ ]:
# Density plot by phenotype score
x = d.obsm['X_umap'][res.kept,0]
y = d.obsm['X_umap'][res.kept,1]

threshold = res.fdr_5p_t
heatmap, xedges, yedges = np.histogram2d(x, y, bins=100, weights = 1*(cell_phen_assoc<-threshold)) #weights
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

plt.clf()
plt.imshow(heatmap.T, extent=extent, origin='lower', cmap = "viridis")
plt.title("Density of Cells Negatively Associated with Phenotype", fontsize=20)
plt.show()

In [ ]:
# Most strongly associated with phenotype
threshold = res.fdr_5p_t
plt.figure(figsize=(10,7))
pos_assoc_cells = np.where(cell_phen_assoc>threshold)[0]
plt.scatter(*d.obsm['X_umap'][res.kept,:].T, c = cell_phen_assoc>threshold, s = 4, alpha = 0.3,
       cmap = 'cividis')
plt.title("Cells Positively Assoc. with Phenotype", fontsize=20)
plt.colorbar()

In [ ]:
# Density plot by phenotype score
x = d.obsm['X_umap'][res.kept,0]
y = d.obsm['X_umap'][res.kept,1]

threshold = res.fdr_5p_t
heatmap, xedges, yedges = np.histogram2d(x, y, bins=100, weights = 1*(cell_phen_assoc>threshold)) #weights
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

plt.clf()
plt.imshow(heatmap.T, extent=extent, origin='lower', cmap = "viridis")
plt.title("Density of Positively-Associated Cells", fontsize=20)
plt.show()

In [ ]:
cell_phen_assoc = CNA_cellscores

In [ ]:
# representation of each of Aparna's clusters among min-fdr positively and negatively associated cells
# (though Laurie's notebook )

aparnaclust = d.obs.aparna2p0.values.copy()

mask = np.abs(res.ncorrs) > res.fdr_5p_t
ac = pd.Series(aparnaclust[res.kept])

print('positive (counts)')
counts = ac[mask & (res.ncorrs>0)].value_counts()
print(counts)
print('proportion of each cluster')
fracs = counts / ac.value_counts()
print(fracs[~np.isnan(fracs)].sort_values(ascending=False))

print('\nnegative (counts)')
counts = ac[mask & (res.ncorrs<0)].value_counts()
print(counts)
print('proportion of each cluster')
fracs = counts / ac.value_counts()
print(fracs[~np.isnan(fracs)].sort_values(ascending=False))

In [ ]:
# Color whole cluster by phenotype score, scatterplot
cluster_sel = 7
plt.figure(figsize=(6,4))
cells_in_clust = np.where(data.obs['aparna2p0_elimsoloclust']==str(cluster_sel))[0]
plt.scatter(*data.obsm['X_umap'][cells_in_clust,].T, c = cell_phen_assoc[cells_in_clust], s = 4, alpha = 0.3,
       cmap = "viridis")
plt.title("Cluster "+str(cluster_sel), fontsize=20)
#plt.xlim(5.5,9)
#plt.ylim(-4.5,-1)
plt.colorbar()

In [ ]:
neg_assoc_cells = np.where(d.obs['negcells'])[0]
pos_assoc_cells = np.where(d.obs['poscells'])[0]

In [ ]:
cluster_sel = 7
np.sum(d.obs['aparna2p0'][res.kept]==cluster_sel)

In [ ]:
# Density plot by phenotype score
cluster_sel = 18
xlim_min = 6
xlim_max = 12
ylim_min = -2
ylim_max = 2
plt.figure(figsize=(6,4))
cells_in_clust = np.where(d.obs['aparna2p0'][res.kept]==cluster_sel)[0]
x = d.obsm['X_umap'][res.kept, :][cells_in_clust,0]
y = d.obsm['X_umap'][res.kept, :][cells_in_clust,1]

heatmap, xedges, yedges = np.histogram2d(x, y, bins=100) #weights
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

plt.clf()
plt.imshow(heatmap.T, extent=extent, origin='lower', cmap = "viridis")
plt.title("Cluster "+str(cluster_sel)+ " All Cells", fontsize=20)
plt.xlim(xlim_min,xlim_max)
plt.ylim(ylim_min,ylim_max)
#plt.colorbar()
plt.show()

plt.figure(figsize=(6,4))
cells_in_clust = np.where(d.obs['aparna2p0']==cluster_sel)[0]
to_show = np.intersect1d(cells_in_clust, pos_assoc_cells)
x = d.obsm['X_umap'][to_show,0]
y = d.obsm['X_umap'][to_show,1]

heatmap, xedges, yedges = np.histogram2d(x, y, bins=100) #weights
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

plt.clf()
plt.imshow(heatmap.T, extent=extent, origin='lower', cmap = "viridis")
plt.title("Cluster "+str(cluster_sel)+ " Top Pos. Cells", fontsize=20)
plt.xlim(xlim_min,xlim_max)
plt.ylim(ylim_min,ylim_max)
#plt.colorbar()
plt.show()

plt.figure(figsize=(6,4))
cells_in_clust = np.where(d.obs['aparna2p0']==cluster_sel)[0]
to_show = np.intersect1d(cells_in_clust, neg_assoc_cells)
x = d.obsm['X_umap'][to_show,0]
y = d.obsm['X_umap'][to_show,1]

heatmap, xedges, yedges = np.histogram2d(x, y, bins=100) #weights
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

plt.clf()
plt.imshow(heatmap.T, extent=extent, origin='lower', cmap = "viridis")
plt.title("Cluster "+str(cluster_sel)+ " Top Neg. Cells", fontsize=20)
plt.xlim(xlim_min,xlim_max)
plt.ylim(ylim_min,ylim_max)
#plt.colorbar()
plt.show()

In [ ]:
# Color whole cluster by membership in top group
threshold = np.percentile(CNA_cellscores, 90)
cluster_sel = 12
plt.figure(figsize=(6,4))
cells_in_clust = np.where(data.obs['leiden2']==str(cluster_sel))[0]
plt.scatter(*data.obsm['X_umap'][cells_in_clust,].T, c = cell_phen_assoc[cells_in_clust]>threshold, s = 4, alpha = 0.3,
       cmap = 'cividis')
plt.title("Cluster "+str(cluster_sel), fontsize=20)
plt.xlim(5.5,9)
plt.ylim(-4.5,-1)
plt.colorbar()

In [ ]:
for cluster_sel in np.unique(data.obs["leiden2"]):
    plt.figure(figsize=(6,4))
    cells_in_clust = np.where(data.obs['leiden2']==str(cluster_sel))[0]
    plt.scatter(*data.obsm['X_umap'][cells_in_clust,].T, c = cell_phen_assoc[cells_in_clust], s = 1, alpha = 0.1,
           cmap = "viridis")
    plt.title("Cluster "+str(cluster_sel), fontsize=20)
    plt.colorbar()

In [ ]:
from scipy.stats import gaussian_kde

In [ ]:
# Density plot of most positively-associated cells
xy = d.obsm['X_umap'][res.kept,:][pos_assoc_cells,].T
z = gaussian_kde(xy)(xy)
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(*d.obsm['X_umap'][res.kept,:][pos_assoc_cells,].T, c=z, s = 5, alpha = 0.6, edgecolor='')
ax.set_xlabel('UMAP1', fontsize = 12)
ax.set_ylabel('UMAP2', fontsize = 12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.title("Density of Cells Most Positively Assoc. with Phenotype",
         fontsize = 15)
plt.show()

In [ ]:
# Density plot of most negatively-associated cells
xy = d.obsm['X_umap'][res.kept,:][neg_assoc_cells,].T
z = gaussian_kde(xy)(xy)
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(*d.obsm['X_umap'][res.kept,:][neg_assoc_cells,].T, c=z, s = 5, alpha = 0.6, edgecolor='')
ax.set_xlabel('UMAP1', fontsize = 12)
ax.set_ylabel('UMAP2', fontsize = 12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.title("Density of Cells Most Negatively Assoc. with Phenotype",
         fontsize = 15)
plt.show()

In [ ]:
import anndata as ad

In [ ]:
pos_assoc_data = ad.AnnData(d.X[res.kept,:][pos_assoc_cells,])
sc.pp.neighbors(pos_assoc_data)
sc.tl.umap(pos_assoc_data)

In [ ]:
data = d

In [ ]:
# Color top 10% cells by cell-type cluster, in original dataset's UMAP
data_df = pd.DataFrame({"X Value": data.obsm['X_umap'][res.kept, :][pos_assoc_cells,0],
                     "Y Value": data.obsm['X_umap'][res.kept, :][pos_assoc_cells,1],
                     "Category": data.obs['leiden2'][res.kept][pos_assoc_cells]})
data_df = data_df.sort_values("Category", ascending=False)
groups = data_df.groupby("Category", sort=False)

fig, ax = plt.subplots(figsize=(10, 6))
ax.margins(0.05) 
palette = np.array(sns.color_palette("bright", len(np.unique(data_df['Category']))))
cc = 0 #color counter
for name, group in groups:
    x_vals = group["X Value"]
    y_vals = group["Y Value"]
    plt.scatter(x_vals, y_vals, marker="o", label=name, s=3, alpha = 0.6,
               c=palette[cc:cc+1,:]) #Used for legend and dots
    cc = cc+1
plt.legend(bbox_to_anchor=(0.85,0.5), loc="center right", fontsize=12, 
           bbox_transform=plt.gcf().transFigure, markerscale=2)
plt.subplots_adjust(left=0.0, bottom=0.1, right=0.6)
ax.set_xlabel('UMAP1', fontsize = 15)
ax.set_ylabel('UMAP2', fontsize = 15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("Cells Most Positively Assoc. with Phenotype",
         fontsize = 18)
plt.grid(color='lightgrey', linestyle='-', linewidth=0.5)
plt.show()

In [ ]:
# Color top 10% cells by cell-type cluster, after independently UMAPped
data_df = pd.DataFrame({"X Value": pos_assoc_data.obsm['X_umap'][:,0],
                     "Y Value": pos_assoc_data.obsm['X_umap'][:,1],
                     "Category": data.obs['leiden2'][res.kept][pos_assoc_cells]})
data_df = data_df.sort_values("Category", ascending=False)
groups = data_df.groupby("Category", sort=False)

fig, ax = plt.subplots(figsize=(10, 6))
ax.margins(0.05) 
palette = np.array(sns.color_palette("bright", len(np.unique(data_df['Category']))))
cc = 0 #color counter
for name, group in groups:
    x_vals = group["X Value"]
    y_vals = group["Y Value"]
    plt.scatter(x_vals, y_vals, marker="o", label=name, s=5, alpha = 0.6,
               c=palette[cc:cc+1,:]) #Used for legend and dots
    cc = cc+1
plt.legend(bbox_to_anchor=(0.85,0.5), loc="center right", fontsize=12, 
           bbox_transform=plt.gcf().transFigure, markerscale=2)
plt.subplots_adjust(left=0.0, bottom=0.1, right=0.6)
ax.set_xlabel('UMAP1', fontsize = 15)
ax.set_ylabel('UMAP2', fontsize = 15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("Cells Most Positively Assoc. with Phenotype",
         fontsize = 18)
plt.grid(color='lightgrey', linestyle='-', linewidth=0.5)
plt.show()

In [ ]:
sc.tl.leiden(pos_assoc_data, resolution=0.2)

In [ ]:
# Color top 10% cells by cell-type cluster, after independently UMAPped and clustered
data_df = pd.DataFrame({"X Value": pos_assoc_data.obsm['X_umap'][:,0],
                     "Y Value": pos_assoc_data.obsm['X_umap'][:,1],
                     "Category": pos_assoc_data.obs['leiden']})
data_df = data_df.sort_values("Category", ascending=False)
groups = data_df.groupby("Category", sort=False)

fig, ax = plt.subplots(figsize=(10, 6))
ax.margins(0.05) 
palette = np.array(sns.color_palette("husl", len(np.unique(data_df['Category']))))
cc = 0 #color counter
for name, group in groups:
    x_vals = group["X Value"]
    y_vals = group["Y Value"]
    plt.scatter(x_vals, y_vals, marker="o", label=name, s=5, alpha = 0.6,
               c=palette[cc:cc+1,:]) #Used for legend and dots
    cc = cc+1
plt.legend(bbox_to_anchor=(0.85,0.5), loc="center right", fontsize=12, 
           bbox_transform=plt.gcf().transFigure, markerscale=2)
plt.subplots_adjust(left=0.0, bottom=0.1, right=0.6)
ax.set_xlabel('UMAP1', fontsize = 15)
ax.set_ylabel('UMAP2', fontsize = 15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("Cells Most Positively Assoc. with Phenotype",
         fontsize = 18)
plt.grid(color='lightgrey', linestyle='-', linewidth=0.5)
plt.show()

In [ ]:
new_clust = 0
compare_clusterings = pd.DataFrame({"Cluster "+str(new_clust): data.obs['leiden2'][res.kept][pos_assoc_cells][np.where(pos_assoc_data.obs['leiden']==str(new_clust))[0]].value_counts()})
for new_clust in np.arange(1, np.max(pos_assoc_data.obs['leiden'].astype(int))+1):
    compare_clusterings["Cluster "+str(new_clust)] = data.obs['leiden2'][res.kept][pos_assoc_cells][np.where(pos_assoc_data.obs['leiden'][res.kept]==str(new_clust))[0]].value_counts()

In [ ]:
compare_clusterings_pct = compare_clusterings/np.sum(compare_clusterings, axis = 0)

In [ ]:
counts = compare_clusterings_pct.transpose()
fig = plt.figure(figsize=(13,7))
ax = fig.add_subplot(111)
counts.plot(ax=ax,kind='bar', stacked=True, rot=0)
vals = ax.get_yticks()
ax.set_yticklabels(['{:3.2f}%'.format(x*100) for x in vals])
ax.yaxis.grid(True)
ax.set_axisbelow(True)
ax.set_xlabel('Clusters Among Top-Scoring Cells', fontsize = 20)
ax.set_ylabel('Fraction of Cells from Each Original Cluster', fontsize = 20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.title("Cells Most Positively Assoc. with Phenotype",
         fontsize = 30)
plt.show()

In [ ]:
neg_assoc_data = ad.AnnData(data.X[neg_assoc_cells,])
sc.pp.neighbors(neg_assoc_data)
sc.tl.umap(neg_assoc_data)

In [ ]:
# Color bottom 10% cells by cell-type cluster, in original dataset's UMAP
data_df = pd.DataFrame({"X Value": data.obsm['X_umap'][neg_assoc_cells,0],
                     "Y Value": data.obsm['X_umap'][neg_assoc_cells,1],
                     "Category": data.obs['leiden2'][neg_assoc_cells]})
data_df = data_df.sort_values("Category", ascending=False)
groups = data_df.groupby("Category", sort=False)

fig, ax = plt.subplots(figsize=(10, 6))
ax.margins(0.05) 
palette = np.array(sns.color_palette("bright", len(np.unique(data_df['Category']))))
cc = 0 #color counter
for name, group in groups:
    x_vals = group["X Value"]
    y_vals = group["Y Value"]
    plt.scatter(x_vals, y_vals, marker="o", label=name, s=3, alpha = 0.6,
               c=palette[cc:cc+1,:]) #Used for legend and dots
    cc = cc+1
plt.legend(bbox_to_anchor=(0.85,0.5), loc="center right", fontsize=12, 
           bbox_transform=plt.gcf().transFigure, markerscale=2)
plt.subplots_adjust(left=0.0, bottom=0.1, right=0.6)
ax.set_xlabel('UMAP1', fontsize = 15)
ax.set_ylabel('UMAP2', fontsize = 15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("Cells Most Negatively Assoc. with Phenotype",
         fontsize = 18)
plt.grid(color='lightgrey', linestyle='-', linewidth=0.5)
plt.show()

In [ ]:
# Color bottom 10% cells by cell-type cluster, after independently UMAPped
data_df = pd.DataFrame({"X Value": neg_assoc_data.obsm['X_umap'][:,0],
                     "Y Value": neg_assoc_data.obsm['X_umap'][:,1],
                     "Category": data.obs['leiden2'][neg_assoc_cells]})
data_df = data_df.sort_values("Category", ascending=False)
groups = data_df.groupby("Category", sort=False)

fig, ax = plt.subplots(figsize=(10, 6))
ax.margins(0.05) 
palette = np.array(sns.color_palette("bright", len(np.unique(data_df['Category']))))
cc = 0 #color counter
for name, group in groups:
    x_vals = group["X Value"]
    y_vals = group["Y Value"]
    plt.scatter(x_vals, y_vals, marker="o", label=name, s=5, alpha = 0.6,
               c=palette[cc:cc+1,:]) #Used for legend and dots
    cc = cc+1
plt.legend(bbox_to_anchor=(0.85,0.5), loc="center right", fontsize=12, 
           bbox_transform=plt.gcf().transFigure, markerscale=2)
plt.subplots_adjust(left=0.0, bottom=0.1, right=0.6)
ax.set_xlabel('UMAP1', fontsize = 15)
ax.set_ylabel('UMAP2', fontsize = 15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("Cells Most Negatively Assoc. with Phenotype",
         fontsize = 18)
plt.grid(color='lightgrey', linestyle='-', linewidth=0.5)
plt.show()

In [ ]:
sc.tl.leiden(neg_assoc_data, resolution=0.2)

In [ ]:
# Color bottom 10% cells by cell-type cluster, after independently UMAPped and clustered
data_df = pd.DataFrame({"X Value": neg_assoc_data.obsm['X_umap'][:,0],
                     "Y Value": neg_assoc_data.obsm['X_umap'][:,1],
                     "Category": neg_assoc_data.obs['leiden']})
data_df = data_df.sort_values("Category", ascending=False)
groups = data_df.groupby("Category", sort=False)

fig, ax = plt.subplots(figsize=(10, 6))
ax.margins(0.05) 
palette = np.array(sns.color_palette("husl", len(np.unique(data_df['Category']))))
cc = 0 #color counter
for name, group in groups:
    x_vals = group["X Value"]
    y_vals = group["Y Value"]
    plt.scatter(x_vals, y_vals, marker="o", label=name, s=5, alpha = 0.6,
               c=palette[cc:cc+1,:]) #Used for legend and dots
    cc = cc+1
plt.legend(bbox_to_anchor=(0.85,0.5), loc="center right", fontsize=12, 
           bbox_transform=plt.gcf().transFigure, markerscale=2)
plt.subplots_adjust(left=0.0, bottom=0.1, right=0.6)
ax.set_xlabel('UMAP1', fontsize = 15)
ax.set_ylabel('UMAP2', fontsize = 15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("Cells Most Negatively Assoc. with Phenotype",
         fontsize = 18)
plt.grid(color='lightgrey', linestyle='-', linewidth=0.5)
plt.show()

In [ ]:
new_clust = 0
compare_clusterings = pd.DataFrame({"Cluster "+str(new_clust): data.obs['leiden2'][neg_assoc_cells][np.where(neg_assoc_data.obs['leiden']==str(new_clust))[0]].value_counts()})
for new_clust in np.arange(1, np.max(neg_assoc_data.obs['leiden'].astype(int))+1):
    compare_clusterings["Cluster "+str(new_clust)] = data.obs['leiden2'][neg_assoc_cells][np.where(neg_assoc_data.obs['leiden']==str(new_clust))[0]].value_counts()
    
    

In [ ]:
compare_clusterings_pct = compare_clusterings/np.sum(compare_clusterings, axis = 0)

In [ ]:
counts = compare_clusterings_pct.transpose()
fig = plt.figure(figsize=(10,7))
ax = fig.add_subplot(111)
counts.plot(ax=ax,kind='bar', stacked=True, rot=0)
vals = ax.get_yticks()
ax.set_yticklabels(['{:3.2f}%'.format(x*100) for x in vals])
ax.yaxis.grid(True)
ax.set_axisbelow(True)
ax.set_xlabel('Clusters Among Top-Scoring Cells', fontsize = 20)
ax.set_ylabel('Fraction of Cells from Each Original Cluster', fontsize = 20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.title("Cells Most Negatively Assoc. with Phenotype",
         fontsize = 30)
plt.show()

In [ ]:
compare_clusterings

In [ ]:
data.obs['leiden2'].value_counts()

In [ ]:
to_export = data.frame("corrs" = res.ncorrs, 
                       "pos_assoc" = res.corrs>threshold, 
                       "neg_assoc" = res.corrs<-threshold)